In [1]:
from haystack.dataclasses import ChatMessage
from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import AzureOpenAIChatGenerator
from haystack import Pipeline
from haystack.utils import Secret

from dotenv import load_dotenv
import os
import json


from prompts import ranking_system_prompt, ranking_user_prompt

In [2]:
# Lade die Umgebungsvariablen aus der .env Datei
load_dotenv()

# Hole die Azure OpenAI Konfiguration aus den Umgebungsvariablen
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15")

In [3]:
titel= "Verstopfter Gulli in meiner Straße"
bild="Nope, kein Bild."
kategorie= "Verstopfter Gulli"
geo_location="52.421, 10.795"
beschreibung="Gulli verstopft, Wasser staut sich. Meine Straße ist überflutet"

In [4]:
prompt_builder = ChatPromptBuilder()
llm = AzureOpenAIChatGenerator(
    api_key=Secret.from_token(azure_api_key),
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment,
    api_version=azure_api_version
)

In [5]:
pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.connect("prompt_builder.prompt", "llm.messages")
messages = [ChatMessage.from_system(ranking_system_prompt),
            ChatMessage.from_user(ranking_user_prompt)]


response = pipe.run(
    data={
        "prompt_builder": {
            "template_variables":{
                "titel": titel, 
                "bild": bild, 
                "kategorie": kategorie, 
                "geo_location": geo_location, 
                "beschreibung": beschreibung
                }, 
            "template": messages
        }
    }
)

result = response["generator"]["replies"][0]


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [12]:
print(result)

NameError: name 'result' is not defined